In [52]:
include("output.jl")
using ImpvolOutput, FileIO
results = load("experiments/baseline/actual/results.jld2")["results"]
parameters = load("experiments/baseline/common_parameters.jld2")["parameters"]
real_GDP_model = sum(ImpvolOutput.make_series(results, :real_GDP), 3)

1×25×1×36 Array{Float64,4}:
[:, :, 1, 1] =
 63700.1  31695.2  38767.1  95719.1  84178.5  …  37380.8  2.04417e5  1.1344e6

[:, :, 1, 2] =
 66184.4  31045.7  41771.9  1.04381e5  …  38399.1  2.12957e5  1.20329e6

[:, :, 1, 3] =
 63191.3  32109.3  41214.8  1.1136e5  …  40223.0  2.07181e5  1.18642e6

...

[:, :, 1, 34] =
 1.58823e5  69293.3  86448.8  2.68306e5  …  74550.2  4.81456e5  3.18149e6

[:, :, 1, 35] =
 1.78912e5  77441.2  91095.8  2.99213e5  …  83378.5  5.32883e5  3.47121e6

[:, :, 1, 36] =
 204441.0  88875.2  1.04067e5  367965.0  …  94990.8  6.19673e5  3.89651e6

In [2]:
volatility1970 = ImpvolOutput.calculate_volatilities(real_GDP_model, parameters, true, 1:8)

1×25×1×1 Array{Float64,4}:
[:, :, 1, 1] =
 0.000420877  0.000217465  0.000659812  …  0.000300208  0.000618529

In [53]:
rows = [("1970s", 1:8), ("1980s", 9:18), ("1990s", 19:28), ("2000s", 28:36)]
columns = ["actual", "kappa1972", "nosectoral", "nosectoral_kappa1972"]

4-element Array{String,1}:
 "actual"              
 "kappa1972"           
 "nosectoral"          
 "nosectoral_kappa1972"

In [54]:
function get_volatility(rows, column)
    results = load("experiments/baseline/$(column)/results.jld2")["results"]
    real_GDP_model = sum(ImpvolOutput.make_series(results, :real_GDP), 3)
    return [mean(ImpvolOutput.calculate_volatilities(real_GDP_model, parameters, true, row[2])) for row in rows]
end

get_volatility (generic function with 1 method)

In [55]:
using DataFrames
volatility = DataFrame()
volatility[:decades] = [row[1] for row in rows]
for column in columns
    volatility[Symbol(column)] = get_volatility(rows, column)
end


In [56]:
volatility

,decades,actual,kappa1972,nosectoral,nosectoral_kappa1972
1,1970s,0.000838944,0.000862658,0.00113311,0.00122769
2,1980s,0.00102563,0.00118439,0.001178,0.00156203
3,1990s,0.00100561,0.0015694,0.00157549,0.00248656
4,2000s,0.000987512,0.0018223,0.00133763,0.00226453


In [57]:
stats = volatility
stats[:trade_barriers] = 100 * (stats[:actual] - stats[:kappa1972]) ./ stats[:kappa1972]
stats[:diversification] = 100 * (stats[:nosectoral] - stats[:nosectoral_kappa1972]) ./ stats[:kappa1972]
stats[:specialization] = 100 * (stats[:actual] - stats[:kappa1972] - stats[:nosectoral] + stats[:nosectoral_kappa1972]) ./ stats[:kappa1972]


4-element Array{Float64,1}:
  8.21461
 19.019  
 22.1283 
  5.05504

In [58]:
for col in columns
    delete!(volatility, Symbol(col))
end

In [59]:
using CSV
CSV.write("volatility_by_decade.csv", volatility)

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "volatility_by_decade.csv", 54, true, String["decades", "trade_barriers", "diversification", "specialization"], 4, false, Val{false})